In [1]:
from pickle import TRUE
import dataSplit
import os, shutil
import redundancy
import Stockholm_Fasta
import PID_save as pid
import redundancy
import numpy as np
import pandas as pd
import mainBlosum
import mainBlosumBrier
import lowerToUpper
import dataCountDescription
import mainNeighbor
import ContextuelBrier

# Initial configuration

Manual initialisation of a folder containing Pfam_dev (sample of the downloaded version from Pfam)

In [2]:
path_main_folder =  "/Users/pauline/Desktop/test_dev" 
name_file_from_Pfam = "Pfam_dev"
path_file_from_Pfam = f"{path_main_folder}/{name_file_from_Pfam}" 


#  User choices

In [3]:
name_folder_stockholm = "Pfam_Stockholm"
name_folder_fasta = "Pfam_fasta"
name_folder_upper = "Pfam_upper"
name_folder_pid = "PID_couple"
name_fasta_non_redondant = "Pfam_fasta_99"

list_residu = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
pid_sup = 99

In [4]:
data_treatment = True
descriptionPfam = True

new_folder = True

blosumGenerator = True
blosum_overfitting_test = True

cube_generator = True



## Data treatment
(do it once !)

In [5]:
path_folder_stockholm = f"{path_main_folder}/{name_folder_stockholm}"  
path_folder_fasta = f"{path_main_folder}/{name_folder_fasta}"
path_folder_upper = f"{path_main_folder}/{name_folder_upper}"
path_folder_pId = f"{path_main_folder}/{name_folder_pid}"
path_folder_fasta_non_redondant =  f"{path_main_folder}/{name_fasta_non_redondant}" 






if data_treatment == True:
    # separation of the Stockholm file into Stockholm files
    Stockholm_Fasta.separationStockholm(path_file_from_Pfam, path_folder_stockholm) 
    #fileNumber.countFile(path_folder_stockholm)  

    # conversion from stockholm into fasta files
    Stockholm_Fasta.multiStockholmToFasta(path_folder_fasta, path_folder_stockholm)  
    #dataCountDescription.dataCountDescription(path_folder_fasta)

    # conversion of all the residus lower case into upper case
    lowerToUpper.multiLowerToUpper(path_folder_fasta, path_folder_upper)
    #dataCountDescription.dataCountDescription(path_folder_upper)

    # pid couple calculation
    pid.savePId(path_folder_upper, path_folder_pId)   

    # redundancy issue  
    redundancy.savePIdNonRedondant(path_folder_upper, path_folder_fasta_non_redondant, pid_sup, list_residu) 
    #dataCountDescription.dataCountDescription(path_folder_fasta_non_redondant)


# description Pfam after data treatment
if descriptionPfam == True:
    dataCountDescription.dataCountDescription(path_folder_fasta_non_redondant, list_residu)




---> Separation of the multiStockholm file into monoStockholm files: 0.01201 s
---> Conversion of Stockholm files into Fasta files: 0.09034 s
---> Correction upper files: 0.01388 s
---> Compute and save the pId files: 2.60851 s
---> Compute and save non-redundant files: 1.16821 s
nbre_seed: 11.00
nbre_seq: 769.00
total_residu: 151,435.00
nbre_position: 2,452.00
mean_len_seq: 196.92
mean_nbre_seq: 69.91
---> Time for data description: 0.54389 s


# Data_split versionning

In [6]:
name_new_folder =  "test_1" # check that the name is not already taken
list_percentage = [50]  
path_new_folder = f"{path_main_folder}/{name_new_folder}"
name_data_A = "Pfam_A"
name_data_B = "Pfam_B"


if new_folder == True:
    if os.path.isdir(path_new_folder ):
        shutil.rmtree(path_new_folder ) 
    os.mkdir(path_new_folder)

    path_folder_fasta_non_redondant =  f"{path_main_folder}/{name_fasta_non_redondant}"   
 
    for percentage_A in list_percentage:
        name_folder_data_split = f"PfamSplit_{str(percentage_A)}"
        path_folder_data_split = f"{path_new_folder}/{name_folder_data_split}"
        dataSplit.dataSplit(path_folder_fasta_non_redondant, path_folder_data_split, percentage_A, name_data_A, name_data_B)

---> Split data_total in data_A and data_B: 0.01209 s


# Non-contextual Blosum

In [7]:
name_BlosumRes = "BlosumRes"
path_BlosumRes = f"{path_new_folder}/{name_BlosumRes}" 




if blosumGenerator == True:
    if os.path.isdir(path_BlosumRes):
        shutil.rmtree(path_BlosumRes) 
    os.mkdir(path_BlosumRes)
    for percentage_train in list_percentage:
        mainBlosum.conditionalProbaGenerator(path_new_folder, percentage_train, path_folder_pId, path_BlosumRes, list_residu, name_data_A)  
        mainBlosum.conditionalProbaGenerator(path_new_folder, percentage_train, path_folder_pId, path_BlosumRes, list_residu, name_data_B) 


folder_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_A
---> Compute the substitution matrix and the conditional probability matrix: 0.21205 s
folder_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_B
---> Compute the substitution matrix and the conditional probability matrix: 1.0503 s


# Brier Score

Over-fitting part

In [8]:
if blosum_overfitting_test == True:
    for percentage_train in list_percentage:
        for data_train in (name_data_A, name_data_B):
            for data_test in (name_data_A, name_data_B):
                mainBlosumBrier.overfittingTest(path_new_folder, percentage_train, path_folder_pId, path_BlosumRes, data_train, data_test, list_residu)

# def overfittingTest(path_data, percentage_A, path_pid, path_BlosumRes, name_data_train, name_data_test):


folder_fasta_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_A
folder_fasta_test: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_A
---> Brier Score with Blosum Predictor: 0.10074 s
Blosum Predictor Brier Score: 0.5912057526322123

folder_fasta_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_A
folder_fasta_test: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_B
---> Brier Score with Blosum Predictor: 0.71656 s
Blosum Predictor Brier Score: 0.6129212118468453

folder_fasta_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_B
folder_fasta_test: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_A
---> Brier Score with Blosum Predictor: 0.25534 s
Blosum Predictor Brier Score: 0.6124667306441905

folder_fasta_train: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_B
folder_fasta_test: /Users/pauline/Desktop/test_dev/test_1/PfamSplit_50/Pfam_B
---> Brier Score with Blosum Predictor: 0.71162 s
Blosum Predictor Brier

# Simple context blosum generator 

(i.e cubes of conditional probabilities)

In [9]:
list_delay_number = [-1, 1]
name_NeighborRes = "NeighborRes"
percentage_train = 50
name_data_train = "Pfam_A"
path_folder_fasta_train = path_folder_fasta_train = f"{path_folder_data_split}/PfamSplit_{str(percentage_train)}/{name_data_train}" 
path_NeighborRes = f"{path_new_folder}/{name_NeighborRes}"


if cube_generator == True:
    for delay_num in list_delay_number:
        for kp_SeqChoice in ["k", "p"]:
            print("{}, {}".format(delay_num, kp_SeqChoice))
            path_proba_cond =  mainNeighbor.simpleContextualBlosum(path_folder_fasta_train, percentage_train, path_folder_pId, path_NeighborRes, delay_num, kp_SeqChoice, pid_inf = 62, scale_factor = 2)
  


NameError: name 'cube_generator' is not defined